In [1]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import os
import cv2

/home/msi/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# make dataset torch format
filenames = os.listdir('dataset/faces/')
path = []
label = []
for i in filenames:
    path.append('dataset/faces/'+i)
    label.append(i.split('.')[0])

# read image
faces = []
for i in path:
    faces.append(cv2.imread(i))

# for each label, make a folder in dataset/torch/
for i in label:
    if not os.path.exists('dataset/torch/'+i):
        os.makedirs('dataset/torch/'+i)
# for each face, save it in dataset/torch/label/
for i in range(len(faces)):
    cv2.imwrite('dataset/torch/'+label[i]+'/'+str(i)+'.jpg', faces[i])


In [3]:
mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('dataset/torch/') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list

In [4]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file

In [5]:
filenames = os.listdir('dataset/ktp/')
path = []
label_ktp = []
for i in filenames:
    path.append('dataset/ktp/'+i)
    label_ktp.append(i.split('.')[0])

# read image
ktps = []
for i in path:
    ktps.append(cv2.imread(i))

In [ ]:
from matplotlib import pyplot as plt
def match(img):
    image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
    face, prob = mtcnn(image, return_prob=True) # returns cropped face and probability
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        dist_list = [] # list of matched distances, minimum distance is used to identify the person
        for idx, emb_db in enumerate(embedding_list):
            dist = torch.dist(emb, emb_db).item()
            dist_list.append(dist)

        if min(dist_list)<1.0:
            idx = dist_list.index(min(dist_list))
            name = name_list[idx]
        else:
            name = 'Unknown'
        return name

img2 = ktps[3].copy()
imgOriginal = img2.copy()
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
name = match(img2)
cv2.putText(imgOriginal, name, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

plt.imshow(cv2.cvtColor(imgOriginal, cv2.COLOR_BGR2RGB))
plt.show()

In [7]:
cap = cv2.VideoCapture(0) # webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break
    img = Image.fromarray(frame)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        dist_list = [] # list of matched distances, minimum distance is used to identify the person
        for idx, emb_db in enumerate(embedding_list):
            dist = torch.dist(emb, emb_db).item()
            dist_list.append(dist)

        if min(dist_list)<1.0:
            idx = dist_list.index(min(dist_list))
            name = name_list[idx]
        else:
            name = 'Unknown'
        cv2.putText(frame, name, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


1.1411741971969604
1.1482659578323364
1.0747711658477783
1.1120469570159912
1.0682789087295532
1.127363920211792
1.1156837940216064
1.1795763969421387
1.122572660446167
1.2935525178909302
1.192293405532837
1.1449308395385742
1.1932129859924316
1.1928086280822754
0.9840807318687439
1.2735880613327026
1.1420841217041016
1.2021286487579346
1.2094988822937012
1.2583197355270386
1.2324340343475342
1.2579737901687622
1.197231411933899
1.138414978981018
1.2405139207839966
1.1664828062057495
1.2200133800506592
1.1610336303710938
1.218800663948059
1.2590293884277344
1.192723274230957
1.176430583000183
1.1867793798446655
1.167521357536316
1.2022712230682373
1.1664541959762573
1.1490607261657715
1.1676924228668213
1.1830558776855469
1.1809747219085693
1.1772420406341553
1.219872236251831
1.2260733842849731
1.23811674118042
1.1614761352539062
1.1328526735305786
0.7726218104362488
0.8000584244728088
0.9086719751358032
0.8528658151626587
0.6588914394378662
0.6529443264007568
0.778224527835846
0.7848